In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/principale/dataprincipale/activia_panache/crop_2_cup_0.42.jpg
/kaggle/input/principale/dataprincipale/activia_panache/crop_0_cup_0.64.jpg
/kaggle/input/principale/dataprincipale/activia_panache/crop_1_cup_0.46.jpg
/kaggle/input/principale/dataprincipale/kids_banane/crop_3_cup_0.05.jpg
/kaggle/input/principale/dataprincipale/oikos_fraise_1/crop_0_boat_0.66.jpg
/kaggle/input/principale/dataprincipale/trefle_ferme_fruits_des_bois/crop_0_cup_0.64.jpg
/kaggle/input/principale/dataprincipale/danao_1l_orange_ananas/crop_0_bottle_0.84.jpg
/kaggle/input/principale/dataprincipale/pot_danette_fraise/crop_0_bowl_0.24.jpg
/kaggle/input/principale/dataprincipale/teen_titan_raven/crop_0_kite_0.06.jpg
/kaggle/input/principale/dataprincipale/teen_titan_robin_verte/crop_0_handbag_0.07.jpg
/kaggle/input/principale/dataprincipale/bouteille_jl_5/crop_0_vase_0.69.jpg
/kaggle/input/principale/dataprincipale/activia_mangue_(1)/crop_0_cup_0.41.jpg
/kaggle/input/principale/dataprincipale/activia_m

In [2]:
!pip install albumentations opencv-python pillow


In [3]:
import os
import cv2
import albumentations as A
import random
import shutil


In [4]:
#  Dataset original (1 image par classe)
INPUT_DIR = "/kaggle/input/principale/dataprincipale"

# Dataset augmenté
AUGMENTED_DIR = "/kaggle/working/dataset_augmente"

#  Dataset final pour YOLO
FINAL_DIR = "/kaggle/working/data_nv"

IMAGES_PAR_CLASSE = 600


In [5]:
A.GaussNoise(p=0.4),  # var_limit par défaut, pas de warning


(GaussNoise(p=0.4, mean_range=(0.0, 0.0), noise_scale_factor=1.0, per_channel=True, std_range=(0.2, 0.44)),)

In [7]:
import albumentations as A

transform = A.Compose([

    # 🔻 1. Dégrader la résolution (très important)
    A.Downscale(
        scale_min=0.2,   # descend à 20% de la taille
        scale_max=0.4,
        interpolation=0,
        p=0.8
    ),

    # 🔻 2. Compression JPEG très agressive
    A.ImageCompression(
        quality_lower=10,
        quality_upper=35,
        p=0.9
    ),

    # 🔻 3. Flou fort
    A.OneOf([
        A.GaussianBlur(blur_limit=(7, 15)),
        A.MotionBlur(blur_limit=15),
        A.MedianBlur(blur_limit=7),
    ], p=0.8),

    # 🔻 4. Bruit important
    A.GaussNoise(
        var_limit=(30.0, 120.0),
        p=0.8
    ),

    # 🔻 5. Mauvaise luminosité
    A.RandomBrightnessContrast(
        brightness_limit=(-0.6, 0.4),
        contrast_limit=(-0.5, 0.3),
        p=0.8
    ),

    # 🔻 6. Petites déformations
    A.Affine(
        scale=(0.7, 1.3),
        shear=(-20, 20),
        rotate=(-45, 45),
        p=0.6
    ),

    # 🔻 7. Taille finale plus petite
    A.Resize(128, 128)

])


/tmp/ipykernel_55/900321814.py:6: UserWarning: Argument(s) 'scale_min, scale_max, interpolation' are not valid for transform Downscale
  A.Downscale(
/tmp/ipykernel_55/900321814.py:14: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(
/tmp/ipykernel_55/900321814.py:28: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(


In [8]:
# Création du dossier de sortie pour les images augmentées
AUGMENTED_DIR = "/kaggle/working/dataset_augmente"
os.makedirs(AUGMENTED_DIR, exist_ok=True)

IMAGES_PAR_CLASSE = 600

for classe in os.listdir(INPUT_DIR):
    classe_path = os.path.join(INPUT_DIR, classe)
    if not os.path.isdir(classe_path):
        continue

    # On prend la seule image présente dans chaque classe
    images = [f for f in os.listdir(classe_path) if f.lower().endswith((".jpg", ".png"))]
    if len(images) == 0:
        print(f"⚠️ Classe vide : {classe}")
        continue

    img_path = os.path.join(classe_path, images[0])
    image = cv2.imread(img_path)
    if image is None:
        print(f"❌ Image non lisible : {img_path}")
        continue

    # Créer le dossier de sortie pour la classe
    out_class_dir = os.path.join(AUGMENTED_DIR, classe)
    os.makedirs(out_class_dir, exist_ok=True)

    print(f"📦 Génération pour la classe : {classe}")

    # Boucle pour générer 500 images
    for i in range(IMAGES_PAR_CLASSE):
        augmented = transform(image=image)
        aug_img = augmented["image"]
        save_path = os.path.join(out_class_dir, f"{classe}_{i:04d}.jpg")
        cv2.imwrite(save_path, aug_img)

    print(f"✅ {IMAGES_PAR_CLASSE} images créées pour {classe}")

print("\n🎉 AUGMENTATION TERMINÉE")


📦 Génération pour la classe : activia_panache
✅ 600 images créées pour activia_panache
📦 Génération pour la classe : kids_banane
✅ 600 images créées pour kids_banane
📦 Génération pour la classe : oikos_fraise_1
✅ 600 images créées pour oikos_fraise_1
📦 Génération pour la classe : trefle_ferme_fruits_des_bois
✅ 600 images créées pour trefle_ferme_fruits_des_bois
📦 Génération pour la classe : danao_1l_orange_ananas
✅ 600 images créées pour danao_1l_orange_ananas
📦 Génération pour la classe : pot_danette_fraise
✅ 600 images créées pour pot_danette_fraise
📦 Génération pour la classe : teen_titan_raven
✅ 600 images créées pour teen_titan_raven
📦 Génération pour la classe : teen_titan_robin_verte
✅ 600 images créées pour teen_titan_robin_verte
📦 Génération pour la classe : bouteille_jl_5
✅ 600 images créées pour bouteille_jl_5
📦 Génération pour la classe : activia_mangue_(1)
✅ 600 images créées pour activia_mangue_(1)
📦 Génération pour la classe : pot_danette_mhalbiya
✅ 600 images créées pou

In [11]:
classes = os.listdir(AUGMENTED_DIR)
print("Nombre de classes :", len(classes))
print("Images dans la première classe :", len(os.listdir(os.path.join(AUGMENTED_DIR, classes[0]))))


Nombre de classes : 80
Images dans la première classe : 600


In [12]:
FINAL_DIR = "/kaggle/working/data_nv"

splits = {
    "train": 0.8,
    "val": 0.1,
    "test": 0.1
}

# Créer les dossiers de split
for split in splits:
    os.makedirs(os.path.join(FINAL_DIR, split), exist_ok=True)

# Distribuer les images dans train / val / test
for classe in os.listdir(AUGMENTED_DIR):
    images = os.listdir(os.path.join(AUGMENTED_DIR, classe))
    random.shuffle(images)

    n = len(images)
    train_end = int(n * 0.8)
    val_end = int(n * 0.9)

    for i, img in enumerate(images):
        if i < train_end:
            split = "train"
        elif i < val_end:
            split = "val"
        else:
            split = "test"

        src = os.path.join(AUGMENTED_DIR, classe, img)
        dst = os.path.join(FINAL_DIR, split, classe)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, os.path.join(dst, img))

print("✅ Dataset final prêt pour YOLO")


✅ Dataset final prêt pour YOLO


In [13]:
import shutil

shutil.make_archive(
    "/kaggle/working/data_nv",  # nom du zip (sans .zip)
    'zip',
    "/kaggle/working",
    "data_nv"
)


'/kaggle/working/data_nv.zip'

In [15]:
import os
import cv2
import albumentations as A
import random
import shutil
import os
import cv2
import albumentations as A
import random
import shutil

# 🔹 Chemins
OLD_DATASET = "/kaggle/working/data_nv"        # dataset original
NEW_DATASET = "/kaggle/working/data_nv_val"    # nouveau dataset avec val augmenté

# 🔹 Supprimer le dataset existant s'il existe
if os.path.exists(NEW_DATASET):
    shutil.rmtree(NEW_DATASET)
os.makedirs(NEW_DATASET, exist_ok=True)

# 🔹 Copier train et test tels quels
for split in ["train", "test"]:
    src_dir = os.path.join(OLD_DATASET, split)
    dst_dir = os.path.join(NEW_DATASET, split)
    shutil.copytree(src_dir, dst_dir)

# 🔹 Définir 10 pipelines différents pour val (10% par pipeline)
pipelines = [
    A.Compose([A.HorizontalFlip(p=0.5), A.Rotate(limit=15, p=0.7), A.Resize(128,128)]),
    A.Compose([A.RandomBrightnessContrast(p=0.7), A.GaussianBlur(blur_limit=7, p=0.7), A.Resize(140,140)]),
    A.Compose([A.GaussNoise(var_limit=(50.0,150.0), p=0.8), A.Resize(130,130)]),
    A.Compose([A.MotionBlur(blur_limit=15, p=0.6), A.Resize(120,120)]),
    A.Compose([A.ImageCompression(quality_lower=5, quality_upper=25, p=0.9), A.Resize(128,128)]),
    A.Compose([A.Downscale(scale_min=(0.1,0.1), scale_max=(0.25,0.25), interpolation=0, p=1.0), A.Resize(140,140)]),
    A.Compose([A.Affine(scale=(0.6,1.2), rotate=(-50,50), shear=(-25,25), p=0.7), A.Resize(125,125)]),
    A.Compose([A.RandomBrightnessContrast(brightness_limit=(-0.6,0.3), contrast_limit=(-0.5,0.3), p=0.8), A.Resize(135,135)]),
    A.Compose([A.GaussianBlur(blur_limit=(9,15), p=0.7), A.GaussNoise(var_limit=(60,180), p=0.7), A.Resize(120,120)]),
    A.Compose([A.ImageCompression(quality_lower=10, quality_upper=35, p=0.8), A.Downscale(scale_min=(0.15,0.15), scale_max=(0.3,0.3), interpolation=0, p=1.0), A.Resize(130,130)])
]

# 🔹 Nouveau val
OLD_VAL = os.path.join(OLD_DATASET, "val")
NEW_VAL = os.path.join(NEW_DATASET, "val")
os.makedirs(NEW_VAL, exist_ok=True)

# 🔹 Parcourir chaque classe
for classe in os.listdir(OLD_VAL):
    class_path = os.path.join(OLD_VAL, classe)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.lower().endswith((".jpg",".png"))]
    random.shuffle(images)

    n = len(images)
    group_size = n // len(pipelines)  # 10% par pipeline
    start_idx = 0

    out_class_dir = os.path.join(NEW_VAL, classe)
    os.makedirs(out_class_dir, exist_ok=True)

    for i, pipeline in enumerate(pipelines):
        end_idx = start_idx + group_size if i < len(pipelines)-1 else n  # dernier groupe prend le reste
        group_images = images[start_idx:end_idx]
        for img_name in group_images:
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            aug = pipeline(image=img)['image']
            save_path = os.path.join(out_class_dir, img_name)
            cv2.imwrite(save_path, aug)
        start_idx = end_idx

print("✅ Nouveau dataset 'data_nv_val' créé avec val fortement dégradé et tailles variables")



/tmp/ipykernel_55/363658284.py:31: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.Compose([A.GaussNoise(var_limit=(50.0,150.0), p=0.8), A.Resize(130,130)]),
/tmp/ipykernel_55/363658284.py:33: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.Compose([A.ImageCompression(quality_lower=5, quality_upper=25, p=0.9), A.Resize(128,128)]),
/tmp/ipykernel_55/363658284.py:34: UserWarning: Argument(s) 'scale_min, scale_max, interpolation' are not valid for transform Downscale
  A.Compose([A.Downscale(scale_min=(0.1,0.1), scale_max=(0.25,0.25), interpolation=0, p=1.0), A.Resize(140,140)]),
/tmp/ipykernel_55/363658284.py:37: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.Compose([A.GaussianBlur(blur_limit=(9,15), p=0.7), A.GaussNoise(var_limit=(60,180), p=0.7), A.Resize(120,120)]),
/tmp/ipykernel_55/363658284.py:38: UserWarning: Argument(s) 'quality_lower, quality_upper' are not 

✅ Nouveau dataset 'data_nv_val' créé avec val fortement dégradé et tailles variables


In [16]:
import shutil
import os

# Dossier à zipper
folder_to_zip = "/kaggle/working/data_nv_val"

# Nom du fichier ZIP
zip_filename = "/kaggle/working/data_nv_val.zip"

# Créer le ZIP
shutil.make_archive(base_name=zip_filename.replace(".zip",""), format='zip', root_dir=folder_to_zip)

print(f"✅ Dataset compressé en : {zip_filename}")


✅ Dataset compressé en : /kaggle/working/data_nv_val.zip
